<a href="https://colab.research.google.com/github/allokkk/Open-AlGym/blob/master/Renforcement_OPenAIgym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reinforcement Learning Tutorial

### Some Common Terms

- Agent
- Environment
- Actions, Rewards, Observations

### 1. Interacting with the Gym API

In [0]:
import gym

In [0]:
# creat gym environment


env = gym.make('CartPole-v0')

#### Each environment comes with certain import methods/attributes
- action_space
- observation_space
- reset() : returns in it state and also resets the environment
- step()
- render()

In [0]:
env.reset()

array([ 0.0066151 , -0.00725424, -0.00368309,  0.03032498])

In [0]:
for t in range(1000):
  env.render()
evn.close( )

NoSuchDisplayException: ignored

In [0]:
for t in range(1000):
  random_action=env.action_space.sample()
  env.step(random_action)# move either left or right

In [0]:
    env.render()

NameError: ignored

In [0]:
env.action_space

Discrete(2)

In [0]:
env.action_space.n

2

In [0]:
env.observation_space.shape[0]

4

## 2. Playing Games with a Random Strategy¶
- Game Episode
- Step() Function in More Detail
- Game Over?

#### step()
- observation
- reward
- done
- Info

In [0]:
for e in range(20): #Episode
    #Play 20 episodes 
    observation = env.reset()#give array observation
    for t in range(50):
        #env.render()
        action = env.action_space.sample()
        observation,reward,done,other_info = env.step(action)
        
        if done:
            # Game Episode is over
            print("Game Episode :{}/{} High Score :{}".format(e,20,t))
            break

env.close()
print("All 20 episodes over!") 

Game Episode :0/20 High Score :11
Game Episode :1/20 High Score :10
Game Episode :3/20 High Score :18
Game Episode :4/20 High Score :9
Game Episode :5/20 High Score :24
Game Episode :6/20 High Score :15
Game Episode :7/20 High Score :11
Game Episode :8/20 High Score :27
Game Episode :9/20 High Score :19
Game Episode :10/20 High Score :31
Game Episode :12/20 High Score :16
Game Episode :13/20 High Score :16
Game Episode :14/20 High Score :21
Game Episode :15/20 High Score :12
Game Episode :16/20 High Score :47
Game Episode :17/20 High Score :17
Game Episode :18/20 High Score :12
Game Episode :19/20 High Score :25
All 20 episodes over!


### 3. Q-Learning

Designing an AI Agent

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import random


Using TensorFlow backend.


In [0]:
class Agent:
    def __init__(self,state_size,action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95 #Discount Factor
        self.epsilon = 1.0 # Exploration Rate: How much to act randomly, 
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001 
        self.model = self._create_model()
        
    
    def _create_model(self):
        #Neural Network To Approximate Q-Value function
        model = Sequential()
        model.add(Dense(24,input_dim=self.state_size,activation='relu')) #1st Hidden Layer
        model.add(Dense(24,activation='relu')) #2nd Hidden Layer
        model.add(Dense(self.action_size,activation='linear'))
        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        return model
    
    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done)) #remembering previous experiences
        
    def act(self,state):
        # Exploration vs Exploitation
        if np.random.rand()<=self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state) # predict reward value based upon current state
        return np.argmax(act_values[0]) #Left or Right
    
    def train(self,batch_size=32): #method that trains NN with experiences sampled from memory
        minibatch = random.sample(self.memory,batch_size)
        for state,action,reward,next_state,done in minibatch:
            
            if not done: #boolean 
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            else:
                target = reward
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state,target_f,epochs=1,verbose=0) #single epoch, x =state, y = target_f, loss--> target_f - 
            
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self,name):
        self.model.load_weights(name)
    def save(self,name):
        self.model.save_weights(name)
            

## Training the DQN Agent(Deep Q-Learner)

In [0]:
n_episodes=1000
output_dir="carpole_model/"

In [0]:
agent = Agent(state_size=4,action_size=2)
done = False
state_size = 4
action_size =2
batch_size = 32

In [0]:
agent = Agent(state_size, action_size) # initialise agent
done = False
for e in range(n_episodes):
    state = env.reset()
    state = np.reshape(state,[1,state_size])
    
    for time in range(5000):
        #env.render()
        action = agent.act(state) #action is 0 or 1
        next_state,reward,done,other_info = env.step(action) 
        reward = reward if not done else -10
        next_state = np.reshape(next_state,[1,state_size])
        agent.remember(state,action,reward,next_state,done)
        state = next_state
        
        if done:
            print("Game Episode :{}/{}, High Score:{},Exploration Rate:{:.2}".format(e,n_episodes,time,agent.epsilon))
            break
            
    if len(agent.memory)>batch_size:
        agent.train(batch_size)
    
    if e%50==0:
        agent.save("output_dir.hdf5")
        
env.close()

Game Episode :0/1000, High Score:59,Exploration Rate:1.0








Game Episode :1/1000, High Score:24,Exploration Rate:0.99
Game Episode :2/1000, High Score:30,Exploration Rate:0.99
Game Episode :3/1000, High Score:10,Exploration Rate:0.99
Game Episode :4/1000, High Score:49,Exploration Rate:0.98
Game Episode :5/1000, High Score:54,Exploration Rate:0.98
Game Episode :6/1000, High Score:43,Exploration Rate:0.97
Game Episode :7/1000, High Score:13,Exploration Rate:0.97
Game Episode :8/1000, High Score:16,Exploration Rate:0.96
Game Episode :9/1000, High Score:34,Exploration Rate:0.96
Game Episode :10/1000, High Score:32,Exploration Rate:0.95
Game Episode :11/1000, High Score:25,Exploration Rate:0.95
Game Episode :12/1000, High Score:22,Exploration Rate:0.94
Game Episode :13/1000, High Score:33,Exploration Rate:0.94
Game Episode :14/1000, High Score:18,Exploration Rate:0.93
Game Episode :15/1000, High Score:20,Exploration Rate:0.93
Game Episode :16/1000, High Score:11,Exploration Rate:0.92
